# Capstone Project - The Battle of the Neighborhoods (Week 1) #
## Applied Data Science Capstone by IBM/Coursera ##
### Fernando Tauscheck ###

## Table of contents:
1. [Introduction: Business Problem](#introduction)<br>
2. [Data](#data)<br>
 1 [Geographic Data:](#data_geolocated)<br>
 2 [Socioeconomic data of the neighborhoods:](#data_socioeconomic)<br>
 3 [Foursquare:](#foursquare)<br>
 4 [Work Flow:](#work_flow)

## 1. Introduction: Business Problem <a name="introduction"></a>

What defines the success of a commercial business? Is location an important factor?

In this project, we will try to understand how location affects a business. Although the analysis can, in theory, be replicated for any type of business, we will focus on stakeholders interested in opening a bakery in Curitiba, Brazil.

Curitiba is the capital and largest city in the Brazilian state of Paraná. The city's population was 1,948,626 as of 2020, making it the eighth most populous city in Brazil and the largest in Brazil's South Region.

According to Foursquare, Curitiba has 608 bakeries and we will try to detect the geographic and social characteristics of the existing bakeries. With this information, we will try to classify the best unexplored points.

As result, we want to provide to our stakeholders a list of possibles location to open a bakery

## 2. Data:  <a name="data"></a>

Some factors will influence our analysis:
* Number of existing bakeries in the neighborhood;
* Socioeconomic data of the neighborhoods (Per capita income, population density, ...);
* Zones from City Master Plan;
* Proximity to parks, public square, boardwalk, main streets, and avenues of great circulation;
* If possible, we will try to compare the Foursquare Rating, Likes, and Tier of each bakery and understand if the location and these pieces of information have any correlation

### 2.1 Geographic Data: <a name="data_geolocated"></a> ###

We will get geographic information from Curitiba at the website of the Instituto de Pesquisa e Planejamento Urbano de Curitiba (Institute of Urban Planning and Research of Curitiba also know as IPPUC)[^1]. The Institute provides all sorts of maps of Curitiba. We will use:

* Zones of City Master Plan;
* Neighborhoods;
* Mains streets;
* Boardwalks, public squares, and parks 

These maps are provided in SHP format (ESRI). Posteriorly they are converted to GeoJSON in a proper representation (WGS84). Then, the GeoJSON files are inserted in an RDMBS (MySQL 8.0), where will be used the Spatial Analysis Functions to analyze. At the GitHub of this project[^2], you can find all support scripts and the structure of the tables.

[^1]: https://ippuc.org.br/geodownloads/geo.htm
[^2]: https://github.com/ftauscheck/The-Battle-of-the-Neighborhoods/tree/main/support


### 2.2 Socioeconomic data of the neighborhoods: <a name="data_socioeconomic"></a> ###
The socioeconomic data of the municipality will be collected from the Wikipedia article: "Lista de bairros de Curitiba"[ˆ3].

[ˆ3]: https://pt.wikipedia.org/wiki/Lista_de_bairros_de_Curitiba

### 2.3 Foursquare: <a name="foursquare"></a> ###

This project will use the Foursquare API as its main data gathering source as it has a database of millions of venues.

To restrict the number of venues to request to API, we will filter only the venus with the category "Bakery". At the same time, to mitigate the problem with neighborhoods with more than 100 bakeries (an API limitation), we will query the API in clusters of hexagons with 1km of radius. With the venues list, an additional request will be made to collect the retrieved details of each venue.

![title](img/01.clusters.png)


### 2.4 Work Flow: <a name="work_flow"></a> ###

With the data collected and processed, we will use a Polynomial Regression algorithm to predict the Rating, Likes, and Tier of each sub-cluster and prepare a list of best clusters to open a bakery. 